# Download the kaggle data set.

In [1]:

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c humpback-whale-identification
!rm -rf input
!mkdir -p input
!unzip -q train.zip -d input/train
#!ls input/train

kaggle.json
  0% 0.00/498k [00:00<?, ?B/s]
100% 498k/498k [00:00<00:00, 69.6MB/s]
  0% 0.00/594k [00:00<?, ?B/s]
100% 594k/594k [00:00<00:00, 82.6MB/s]
 99% 1.33G/1.35G [00:10<00:00, 147MB/s]
100% 1.35G/1.35G [00:10<00:00, 136MB/s]
100% 4.15G/4.16G [00:32<00:00, 148MB/s]
100% 4.16G/4.16G [00:32<00:00, 135MB/s]


In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split


from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)


Using TensorFlow backend.


# Explore the training data.
Let us explore the training data.


In [3]:
train_df = pd.read_csv("train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [4]:
train_df.Id.value_counts()

new_whale    9664
w_23a388d      73
w_9b5109b      65
w_9c506f6      62
w_0369a5c      61
w_700ebb4      57
w_3de579a      54
w_564a34b      51
w_fd3e556      50
w_88e4537      49
w_2b069ba      48
w_d405854      47
w_789c969      45
w_f0fe284      45
w_343f088      40
w_5e8e218      40
w_778e474      40
w_a9304b9      37
w_60ce6fc      37
w_5a2634c      37
w_6822dbc      36
w_af367c3      35
w_f765256      34
w_1ca9ab1      34
w_17b0d3a      33
w_d72771c      32
w_08630fd      31
w_8c25681      31
w_6cda039      31
w_fd6a3ad      30
             ... 
w_d240f68       1
w_cdc55a8       1
w_f94d9bb       1
w_ba668da       1
w_e39f7d5       1
w_d869993       1
w_8ace2d9       1
w_59f2171       1
w_e8b2157       1
w_9af8eba       1
w_169253a       1
w_7d34a32       1
w_3bf887a       1
w_832917d       1
w_b230d25       1
w_8b05f21       1
w_9e5c031       1
w_fe9eac7       1
w_81ee2ec       1
w_529fa56       1
w_c49a22d       1
w_a68cf64       1
w_e0ca4f5       1
w_94c3bf0       1
w_c67271f 

In [5]:
print('Number of rows in train.csv', len(train_df))

Number of rows in train.csv 25361


# Identify the data points

## train.csv
There are 25361 rows in train.csv.  Which corresponds to the image entries in train.zip
We can see that the train.csv file has two data fields.  
* Image : The whale image file name
* Id is the whale Id.
Each whale is assigned a unique Id.  The unidentified whale's are assigned an Id new_whale.  


## train.zip
There are 25361 image files in train.zip file.  It has been extracted to input/train folder.  The filename corresponds to the Image column in train.csv file.